In [11]:
import pandas as pd
from sqlalchemy import create_engine
from functions import list_countries_un


In [12]:
import pymysql
pymysql.install_as_MySQLdb()

<strong>Solar/Electricity: </strong>

In [15]:
un_solar_csv = "Resources/UNdata_total_electricity.csv"
df = pd.read_csv(un_solar_csv)
df.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Afghanistan,Electricity - Gross production,2017.0,"Kilowatt-hours, million",1098.2,1.0
1,Afghanistan,Electricity - Gross production,2016.0,"Kilowatt-hours, million",1076.3,1.0
2,Afghanistan,Electricity - Gross production,2015.0,"Kilowatt-hours, million",1033.7,1.0
3,Afghanistan,Electricity - Gross production,2014.0,"Kilowatt-hours, million",1049.3,1.0
4,Afghanistan,Electricity - Gross production,2013.0,"Kilowatt-hours, million",1022.3,1.0


In [16]:
electric_country_list = list_countries_un(df)

In [17]:
electric_country_list.head()

,country_name
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra


In [7]:
rds_connection_string = "root:64535864@localhost/energy_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [8]:
engine.table_names()

['country', 'solar_energy']

In [18]:
electric_country_list.to_sql('country', con=engine, if_exists='append', index=False)

NameError: name 'engine' is not defined

In [19]:
un_electic_df = df[['Country or Area', 'Commodity - Transaction', 'Year', 'Unit', 'Quantity']]

In [21]:
un_electic_df = un_electic_df.rename(columns={
    'Country or Area': 'country',
    'Commodity - Transaction': 'commodity_transaction',
    'Year': 'year',
    'Unit': 'unit',
    'Quantity': 'quantity'
})

In [22]:
un_electic_df = un_electic_df.dropna()

In [12]:
un_electic_df.to_sql(name='un_total_electric_energy', con=engine, if_exists='append', index=False)

In [13]:
eia_electric_csv = 'Resources/electric.csv'

In [14]:
df_electric = pd.read_csv(eia_electric_csv)

In [17]:
df_electric.head()

,"Table 5.4.B. Sales of Electricity to Ultimate Customers by End-Use Sector,",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,"by State, Year-to-Date through November 2018 a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Residential,NaN,Commercial,NaN,Industrial,NaN,Transportation,NaN,All Sectors,NaN
2,Census Division\nand State,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD
3,New England,"43,722","41,436","48,097","47,818","14,751","15,508",520,502,"107,090","105,264"
4,Connecticut,"11,882","11,141","11,391","11,313","2,836","2,993",178,162,"26,287","25,609"


In [35]:
df_electric['country'] = 'United States'

In [94]:
renamed_df_electric = df_electric.rename(columns={
    'Table 5.4.B. Sales of Electricity to Ultimate Customers by End-Use Sector,': 'division_state',
    'Unnamed: 1': 'resident_nov_2018',
    'Unnamed: 2': 'resident_nov_2017',
    'Unnamed: 3': 'comm_nov_2018',
    'Unnamed: 4': 'comm_nov_2017',
    'Unnamed: 5': 'indust_nov_2018',
    'Unnamed: 6': 'indust_nov_2017',
    'Unnamed: 7': 'trans_nov_2018',
    'Unnamed: 8': 'trans_nov_2017',
    'Unnamed: 9': 'all_sect_nov_2018',
    'Unnamed: 10': 'all_sect_nov_2017',
})

In [95]:
renamed_df_electric.head()

,division_state,resident_nov_2018,resident_nov_2017,comm_nov_2018,comm_nov_2017,indust_nov_2018,indust_nov_2017,trans_nov_2018,trans_nov_2017,all_sect_nov_2018,all_sect_nov_2017,country
0,"by State, Year-to-Date through November 2018 a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States
1,NaN,Residential,NaN,Commercial,NaN,Industrial,NaN,Transportation,NaN,All Sectors,NaN,United States
2,Census Division\nand State,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,United States
3,New England,"43,722","41,436","48,097","47,818","14,751","15,508",520,502,"107,090","105,264",United States
4,Connecticut,"11,882","11,141","11,391","11,313","2,836","2,993",178,162,"26,287","25,609",United States


In [96]:
new_df_electric = renamed_df_electric[['country', 'division_state', 'resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']].iloc[3:]
new_df_electric.head()

,country,division_state,resident_nov_2018,resident_nov_2017,comm_nov_2018,comm_nov_2017,indust_nov_2018,indust_nov_2017,trans_nov_2018,trans_nov_2017,all_sect_nov_2018,all_sect_nov_2017
3,United States,New England,"43,722","41,436","48,097","47,818","14,751","15,508",520,502,"107,090","105,264"
4,United States,Connecticut,"11,882","11,141","11,391","11,313","2,836","2,993",178,162,"26,287","25,609"
5,United States,Maine,"4,240","4,217","3,674","3,623","2,576","2,456",0,0,"10,490","10,296"
6,United States,Massachusetts,"18,604","17,528","23,723","23,759","5,594","6,287",317,315,"48,238","47,888"
7,United States,New Hampshire,"4,203","3,998","4,067","4,012","1,800","1,806",0,0,"10,069","9,817"


In [97]:
# method to replace ',' with ''
new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']] = new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']].apply(lambda x: x.str.replace(',',''))



In [98]:
# convert to numeric
new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']] = new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']].apply(pd.to_numeric)


In [105]:
new_df_electric = new_df_electric.dropna()


In [107]:
new_df_electric.to_sql(name='us_state_electric', con=engine, if_exists='append', index=False)

<strong>Natural Gas: </strong>

In [ ]:
csv_file1 = "Resources/UNdata_Export_NaturalGasConsumption.csv"
Natural_Gas_Consumption_UN_df = pd.read_csv(csv_file1)
Natural_Gas_Consumption_UN_df.head()

In [ ]:
natural_gas_country_list = list_countries_un(Natural_Gas_Consumption_UN_df)
natural_gas_country_list.count()

In [ ]:
natural_gas_country_list.to_sql('country', con=engine, if_exists='append', index=False)

In [ ]:
#Drop Column 'Quantity Footnotes'
Natural_Gas_Consumption_UN_df.drop(Natural_Gas_Consumption_UN_df.columns[[5]], axis=1, inplace=True)
Natural_Gas_Consumption_UN_df.head()

In [ ]:
#Rename columns
Natural_Gas_Consumption_UN_df.columns = ['Country', 'Commodity_Transaction', 'Year', 'Unit', 'Quantity']
Natural_Gas_Consumption_UN_df.head()

In [ ]:
engine.table_names()

In [ ]:
Natural_Gas_Consumption_UN_df.to_sql(name='EnergyConsumptionUN', con=engine, if_exists='append', index=False)

In [ ]:
csv_file1 = "Resources/Total_consumption_Natural_Gas_EIA2.csv"
Natural_Gas_Consumption_EIA_df = pd.read_csv(csv_file1)
Natural_Gas_Consumption_EIA_df.head()

In [ ]:
#Delete Country = Production rows
Natural_Gas_Consumption_EIA_df = Natural_Gas_Consumption_EIA_df.dropna()
Natural_Gas_Consumption_EIA_df.head()

In [ ]:
Natural_Gas_Consumption_EIA_df.columns = ['Year_' + str(col) for col in Natural_Gas_Consumption_EIA_df.columns]
Natural_Gas_Consumption_EIA_df.head()

In [ ]:
engine.table_names()

In [ ]:
#Use pandas to load csv concerted to dataFrame into database
Natural_Gas_Consumption_EIA_df.to_sql(name='EnergyConsumptionEIA', con=engine, if_exists='append', index=False)